In [21]:
# Imports
import json, os
import pandas as pd
from yelpapi import YelpAPI

In [22]:
# Load Credentials
with open('/Users/julia/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [23]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [24]:
# Set our API call parameters and filename before the first call
LOCATION = 'San Diego, CA'
TERM = 'Pasta'

In [25]:
# Specifying JSON_FILE filename 
JSON_FILE = f"Data/results_in_progress_pasta.json"
JSON_FILE

'Data/results_in_progress_pasta.json'

In [26]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: 
if file_exists == False:
    
    # CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    # INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    # save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_pasta.json already exists.


In [27]:
# Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [28]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [29]:
# How many results total?
total_results = results['total']
total_results

956

In [30]:
# How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [31]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

48

In [35]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/48 [00:00<?, ?it/s]

In [36]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)



  0%|          | 0/48 [00:00<?, ?it/s]

In [37]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,W0dzJ7E6R3kyAOXIlO_tbw,ristorante-illando-san-diego,Ristorante Illando,https://s3-media3.fl.yelpcdn.com/bphoto/7U-OZH...,False,https://www.yelp.com/biz/ristorante-illando-sa...,2140,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 32.724048614502, 'longitude': -11...","[restaurant_reservation, delivery, pickup]",$$,"{'address1': '1825 India Street', 'address2': ...",+16196935204,(619) 693-5204,7514.018555
1,ruZ45_rMndqb7aLEr6VSOw,cesarina-san-diego-3,Cesarina,https://s3-media1.fl.yelpcdn.com/bphoto/Yv3Z0k...,False,https://www.yelp.com/biz/cesarina-san-diego-3?...,1402,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",5.0,"{'latitude': 32.74319, 'longitude': -117.23453}",[delivery],$$,"{'address1': '4161 Voltaire St', 'address2': '...",+16192266222,(619) 226-6222,9184.829871
2,qXRzQERT-WFDo6spPeQwZw,pesto-italian-craft-kitchen-mission-valley-san...,Pesto Italian Craft Kitchen - Mission Valley,https://s3-media2.fl.yelpcdn.com/bphoto/OVbDWc...,False,https://www.yelp.com/biz/pesto-italian-craft-k...,345,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 32.769922, 'longitude': -117.146673}","[delivery, pickup]",$$,"{'address1': '1652 Camino Del Rio N', 'address...",+16196799030,(619) 679-9030,2395.158360
3,4GT1sQ_pcKk4gBMrJddexw,pesto-italian-craft-kitchen-college-san-diego-2,Pesto Italian Craft Kitchen College,https://s3-media3.fl.yelpcdn.com/bphoto/rFQAlX...,False,https://www.yelp.com/biz/pesto-italian-craft-k...,1730,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 32.760629, 'longitude': -117.067999}","[delivery, pickup]",$$,"{'address1': '6011 El Cajon Blvd A', 'address2...",+16192652700,(619) 265-2700,8709.615928
4,AlO78to1upUkVhRx12XtVA,cucina-basilico-san-diego,Cucina Basilico,https://s3-media4.fl.yelpcdn.com/bphoto/iiExW4...,False,https://www.yelp.com/biz/cucina-basilico-san-d...,672,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 32.81065, 'longitude': -117.116113}",[delivery],$$,"{'address1': '3755 Murphy Canyon Rd', 'address...",+18588746436,(858) 874-6436,4192.916038


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
951,j_te1-9iaR9wKninc321ZA,jimbos-carlsbad,Jimbo's,https://s3-media3.fl.yelpcdn.com/bphoto/Bw6NSw...,False,https://www.yelp.com/biz/jimbos-carlsbad?adjus...,221,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.0,"{'latitude': 33.0723073917475, 'longitude': -1...",[delivery],$$,"{'address1': '1923 Calle Barcelona', 'address2...",+17603347755,(760) 334-7755,32993.906277
952,_MUDy_IRMbmW9qoBbGkfmg,dominos-pizza-escondido-3,Domino's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/ELK2kM...,False,https://www.yelp.com/biz/dominos-pizza-escondi...,110,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 33.1343765258789, 'longitude': -1...",[],$,"{'address1': '120 E Lincoln Ave', 'address2': ...",+17604800800,(760) 480-0800,38748.621683
953,UoZgAW9b5XTP_xMMBnfDwA,panera-bread-san-diego-5,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/XW_qki...,False,https://www.yelp.com/biz/panera-bread-san-dieg...,214,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 33.019965, 'longitude': -117.114366}","[delivery, pickup]",$$,"{'address1': '10430 Craftsman Way', 'address2'...",+18584511500,(858) 451-1500,25775.678988
954,gVhkehhbDCxyd2K1soQHqQ,pizza-hut-san-marcos-13,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/0MPuzW...,False,https://www.yelp.com/biz/pizza-hut-san-marcos-...,54,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 33.132928, 'longitude': -117.121677}","[delivery, pickup]",$,"{'address1': '705 Center Dr', 'address2': 'Ste...",+17604896000,(760) 489-6000,38186.255621
955,CO7imJydnHW1wyFP6ZPZqA,panera-bread-solana-beach,Panera Bread,https://s3-media3.fl.yelpcdn.com/bphoto/ptgZ5u...,False,https://www.yelp.com/biz/panera-bread-solana-b...,167,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.0,"{'latitude': 32.99386433036879, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '667 San Rodolfo Drive', 'address...",+18584810050,(858) 481-0050,24647.555727


In [38]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

1

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('Data/results_Toronto_Sushi.csv.gz', compression='gzip',index=False)